# `01` Import Necessary Libraries

In [114]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [ ]:
movie_data = pd.merge(ratings, movies, on='movieId')


In [ ]:

movies["year"]=movie_data.title.str.extract(r"\((\d{4})\)", expand=True)
movies["year"].fillna(1995, inplace=True)
movies["year_cat"]=pd.cut(movies["year"].astype(int), bins=[1900, 1970, 1990, 2000, 2010, 2020], labels=["1900-1970", "1970-1990", "1990-2000", "2000-2010", "2010-2020"])
year_cat=pd.get_dummies(movies["year_cat"],dtype=int)   
year_cat["movieId"]=movies["movieId"] 

In [ ]:
  
movie_genres = movies.set_index('movieId')['genres'].str.get_dummies(sep='|')


In [ ]:
utility_matrix= movie_data.pivot_table(index='movieId', columns='userId', values='rating')

In [ ]:
utility_matrix.fillna(0, inplace=True)


In [ ]:

final=utility_matrix.merge(movie_genres, on='movieId', how='inner')
final=final.merge(year_cat, on='movieId', how='inner')
final.set_index('movieId', inplace=True)


In [ ]:
similarity=cosine_similarity(final)
similarity=pd.DataFrame(similarity, index=final.index, columns=final.index,dtype=float)


In [148]:
def get_similar_movies(movieId):
    sim=similarity[movieId].sort_values(ascending=False).head(6).index[1:]
    return movies[movies['movieId'].isin(sim)]
get_similar_movies(1)

,movieId,title,genres,year,year_cat
224,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,1981,1970-1990
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War,1995,1990-2000
418,480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,1998,1990-2000
615,780,Independence Day (a.k.a. ID4) (1996),Action|Adventure|Sci-Fi|Thriller,1996,1990-2000
2355,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,1994,1990-2000
